In [1]:
import pyarrow.parquet as pq
import torch
import random
import time
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [14]:
temp = pq.read_table('./Data/cleaned_taxi_data.parquet')
temp = temp.to_pandas()

trip_inf_loc = list(range(0, 5)) + list(range(6, 172))
tip_inf_loc = [172, 173, 174, 175] 
trip_info = torch.tensor(temp.iloc[:, trip_inf_loc].values)
y = torch.tensor(temp.iloc[:, tip_inf_loc].values)


In [4]:
# Base values
RANDOM_SEED = 123
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 64
NUM_EPOCHS = 50

In [5]:
# Normalize and Split Input Data (Train = 75% (1,707,346), Validation = 20% (455,292), Test = 5% (113,823))
torch.manual_seed(RANDOM_SEED)

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(trip_info,  [0.75, 0.2, 0.05])


In [16]:

class MLP(torch.nn.Module):

    def __init__(self, num_features, num_hidden_1, num_classes):
        super(MLP, self).__init__()
        
        self.num_classes = num_classes
        
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        self.bn1 = torch.nn.BatchNorm1d(self.linear_1.weight.size(0))
        
    def forward(self, x):
        
        ### MAKE SURE YOU CONNECT THE LAYERS PROPERLY IF YOU CHANGED
        ### ANYTHNG IN THE __init__ METHOD ABOVE       
        out = self.linear_1(x)
        out = self.bn1(out)
        out = F.relu(out)
        logits = self.linear_out(out)
        probas = F.softmax(logits, dim=1)
        return logits, probas
    
#################################
### Model Initialization
#################################


random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

model = MLP(num_features=171,
            num_hidden_1=100,
            num_classes=4)


model = model.to(DEVICE)


In [31]:
print(train_dataset.dataset[0].view(-1, 171))

tensor([[ 1.0000,  0.9000,  5.0000,  3.0000,  0.5000,  0.0000,  0.3000, 11.0000,
          2.5000,  0.0000,  0.2000,  1.0000,  0.0000, 13.0000,  1.0000,  0.0000,
         17.0000, 67.4000, 49.9000, 56.5000,  5.8000, 30.2000,  0.0000,  1.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  

In [8]:

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

train_acc_lst, valid_acc_lst = [], []
train_loss_lst, valid_loss_lst = [], []
#train_dataset, val_dataset, test_dataset, y
for epoch in range(NUM_EPOCHS):
    
    model.train()
    
    for i in range(y.shape()[0]):
        
        ### PREPARE MINIBATCH
        features = train_dataset.dataset[i].view(-1, 171).to(DEVICE)
        targets = y.to(DEVICE)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        #if not batch_idx % 20:
        #    print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
        #           f'Batch {batch_idx:03d}/{len(train_loader):03d} |' 
        #           f' Cost: {cost:.4f}')

    # no need to build the computation graph for backprop when computing accuracy
    model.eval()
    
    with torch.set_grad_enabled(False):
        train_acc, train_loss = compute_accuracy_and_loss(model, train_loader, device=DEVICE)
        valid_acc, valid_loss = compute_accuracy_and_loss(model, valid_loader, device=DEVICE)
        train_acc_lst.append(train_acc)
        valid_acc_lst.append(valid_acc)
        train_loss_lst.append(train_loss)
        valid_loss_lst.append(valid_loss)
        print(f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} Train Acc.: {train_acc:.2f}%'
              f' | Validation Acc.: {valid_acc:.2f}%')
        
    elapsed = (time.time() - start_time)/60
    print(f'Time elapsed: {elapsed:.2f} min')
  
elapsed = (time.time() - start_time)/60
print(f'Total Training Time: {elapsed:.2f} min')

ValueError: too many values to unpack (expected 2)